In [ ]:

import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from datetime import datetime

class ExpenseTracker:
    def _init_(self, root):
        self.root = root
        self.root.title("Expense Tracker with Prediction")
        self.root.geometry("800x600")
        
        # Initialize database
        self.init_database()
        
        # Create GUI
        self.create_widgets()
        
        # Load expenses
        self.load_expenses()
    
    def init_database(self):
        """Database Connectivity - Create SQLite database and table"""
        self.conn = sqlite3.connect('expenses.db')
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS expenses (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                category TEXT,
                amount REAL,
                description TEXT
            )
        ''')
        self.conn.commit()
    
    def create_widgets(self):
        """Tkinter - Create GUI elements"""
        # Title
        title = tk.Label(self.root, text="Expense Tracker", font=("Arial", 20, "bold"), fg="#2c3e50")
        title.pack(pady=10)
        
        # Input Frame
        input_frame = tk.Frame(self.root, bg="#ecf0f1")
        input_frame.pack(pady=10, padx=20, fill="x")
        
        # Category
        tk.Label(input_frame, text="Category:", bg="#ecf0f1").grid(row=0, column=0, padx=5, pady=5)
        self.category_var = tk.StringVar()
        self.category_combo = ttk.Combobox(input_frame, textvariable=self.category_var, width=15)
        self.category_combo['values'] = ('Food', 'Transport', 'Entertainment', 'Bills', 'Shopping', 'Other')
        self.category_combo.grid(row=0, column=1, padx=5, pady=5)
        
        # Amount
        tk.Label(input_frame, text="Amount:", bg="#ecf0f1").grid(row=0, column=2, padx=5, pady=5)
        self.amount_entry = tk.Entry(input_frame, width=15)
        self.amount_entry.grid(row=0, column=3, padx=5, pady=5)
        
        # Description
        tk.Label(input_frame, text="Description:", bg="#ecf0f1").grid(row=1, column=0, padx=5, pady=5)
        self.desc_entry = tk.Entry(input_frame, width=40)
        self.desc_entry.grid(row=1, column=1, columnspan=3, padx=5, pady=5)
        
        # Buttons Frame (placeholder for now)
        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=10)
        
        # Treeview for displaying expenses
        tree_frame = tk.Frame(self.root)
        tree_frame.pack(pady=10, padx=20, fill="both", expand=True)
        
        self.tree = ttk.Treeview(tree_frame, columns=("Date", "Category", "Amount", "Description"), show="headings")
        self.tree.heading("Date", text="Date")
        self.tree.heading("Category", text="Category")
        self.tree.heading("Amount", text="Amount")
        self.tree.heading("Description", text="Description")
        
        self.tree.column("Date", width=100)
        self.tree.column("Category", width=100)
        self.tree.column("Amount", width=80)
        self.tree.column("Description", width=200)
        
        scrollbar = ttk.Scrollbar(tree_frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Total Label
        self.total_label = tk.Label(self.root, text="Total Expenses: $0.00", font=("Arial", 14, "bold"), fg="#e74c3c")
        self.total_label.pack(pady=10)
    
    def load_expenses(self):
        """Load expenses from database"""
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        self.cursor.execute("SELECT date, category, amount, description FROM expenses ORDER BY date DESC")
        expenses = self.cursor.fetchall()
        
        total = 0
        for expense in expenses:
            self.tree.insert("", "end", values=expense)
            total += expense[2]
        
        self.total_label.config(text=f"Total Expenses: ${total:.2f}")
    
    def _del_(self):
        """Close database connection"""
        self.conn.close()

if _name_ == "_main_":
    root = tk.Tk()
    app = ExpenseTracker(root)
    root.mainloop()

